In [ ]:
!pip install -U duckduckgo_search
# Doc: https://pypi.org/project/duckduckgo-search/

In [ ]:
!pip install newspaper3k

In [ ]:
from duckduckgo_search import DDGS
from newspaper import Article
import json

In [ ]:
from duckduckgo_search import ddg

In [ ]:
with DDGS() as ddgs:
    keywords = '"huracán otis"'
    ddgs_news_gen = ddgs.text(
      keywords,
      region="mx-es",
      safesearch="off",
      backend='lite',
      timelimit='m',
      max_results=500
    )
    
urls = []  # Inicializamos una lista vacía para almacenar las URLs
for r in ddgs_news_gen:
    urls.append(r['href'])

In [17]:
results = ddg('"huracán otis"', max_results=100)
urls = [result['href'] for result in results]

In [18]:
urls

['https://www.cnn.com/videos/spanish/2023/11/03/acapulco-otis-devastacion-terror-orix-perspectivas-mexico.cnn',
 'https://www.bbc.com/mundo/articles/cjl03ep04yyo',
 'https://www.bbc.com/mundo/articles/c807nr77p4xo',
 'https://www.cnn.com/videos/spanish/2023/10/25/otis-huracan-acapulco-guerrero-perspectivas-mexico-tv.cnn',
 'https://www.nytimes.com/es/2023/10/25/espanol/huracan-otis-mexico-acapulco.html',
 'https://www.cnn.com/videos/spanish/2023/10/25/hurcan-otis-acapulco-orix.cnn',
 'https://cnnespanol.cnn.com/2023/10/25/tormenta-tropical-otis-trayectoria-a-donde-se-dirige-guerrero-oaxaca-orix/',
 'https://www.bloomberg.com/news/articles/2023-10-25/huracan-otis-azota-mexico-tras-tocar-tierra-cerca-de-acapulco',
 'https://ciencia.nasa.gov/ciencias-terrestres/el-huracan-otis/',
 'https://cnnespanol.cnn.com/2023/10/27/huracan-otis-danos-noticias-acapulco-mexico-orix-2/',
 'https://cnnespanol.cnn.com/2023/10/25/mira-fotos-llegada-huracan-otis-mexico-orix/',
 'https://es.wikipedia.org/wiki

In [ ]:
def extract_content_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return {
            "title": article.title,
            "content": article.text,
            "url": url,
            "date": article.publish_date.strftime('%Y-%m-%d') if article.publish_date else None,
            "image_url": article.top_image
        }
    except Exception as e:
        print(f"Error al extraer contenido de {url}. Error: {e}")
        return None

In [ ]:
import os

# Definimos la ruta del archivo
archivo_path = './02_news/noticias_huracan_otis.jsonl'

# Diccionario para almacenar las noticias, usando la URL como llave
noticias_dict = {}

# Si el archivo ya existe, lo leemos para cargar las noticias existentes
if os.path.exists(archivo_path):
    with open(archivo_path, 'r', encoding='utf-8') as file:
        for line in file:
            noticia = json.loads(line.strip())
            noticias_dict[noticia['url']] = noticia

# Procesamos las URLs nuevas
for r in urls:
    print(r)
    if r not in noticias_dict:  # Verificamos que la URL no exista ya en el diccionario
        content = extract_content_from_url(r)
        if content:
            noticias_dict[content['url']] = content  # Agregamos la noticia al diccionario

# Guardamos las noticias (tanto las antiguas como las nuevas) en el archivo
with open(archivo_path, 'w', encoding='utf-8') as file:
    for noticia in noticias_dict.values():
        file.write(json.dumps(noticia, ensure_ascii=False) + '\n')

print("¡Proceso completado!")
